# Data Rangers: Ingenieria de datos

## 1. Base de datos

- La base de datos se desplego localmente, en la ultima imagen de docker de ```postgresql```. Como se detalla en el ```README.md```. Si se cuenta con una base de datos inicial se pueden reemplazar los valores en el archivo ```.env```

- En este caso se asume que las tablas pueden o no estar creada dentro de la empresa, de ahi que se usan los objetos relacionales de sqlalchemy para mapear estas tablas con sus columnas y esquemas.

## 2. Pipeline

Para este pipeline en particular simulando procesamiento cercano al tiempo real, es importante que los archivos no se almacenen en memoria. Para ello, se utilizan generadores, que al final terminan siendo objetos (mucho mas reducidos comparado al peso del archivo).  
Estos objetos van a arrojar un iterable que luego vamos a poder acceder siempre que se necesite. Es similar al lazy-evaluation de ``` apache spark```.  

En ese sentido, se utilizo la palabra clave ```yield``` y se accedio al archivo solo cuando se necesita.  Adicional, se aprovecho la funcionalidad de la API de Pandas para generar chunks, estos chunks tambien funcionan creando un objeto iterable que luego podemos acceder cuando se necesita y asi realizar la ingesta.  

#### Este pipeline tiene las siguientes ventajas:
- Carga informacion solo cuando se necesita.
- Procesa cada fila de forma individual.
- Parcialmente utiliza el ORM de sqlalchemy lo que lo hace bastante sencillo luego de integrar con API`s basadas en ```Flask```,```Django``` o ```Fast API``` o en su defecto, utilizando el ORM propio de estos frameworks. 

#### A su vez, tiene estos puntos por mejorar:
- Configurar de tal manera que el motor de base de datos pueda ser determinado por el usuario que programa el pipeline.
- Insertar pruebas unitarias
- Utilizar frameworks mas avanzados en procesamiento NRT (near-real-time) como ```apache akafka```.

### 2.1. Paquetes necesarios

In [ ]:
import utils.dbconfig as dbc # Modulo de configuraciones en carpeta utils

from utils.functions   import list_datafile # Funcion para listar archivos

from utils.logs        import PipelineLogs  # Clase para los logs
from database.database import Database      # Clase para conexion a base de datos

from etl_pipeline      import pipeline

In [ ]:
## Variables de entorno:
USER = dbc.DB_USER
PWD  = dbc.DB_PASSWORD
HOST = dbc.DB_HOST
DB   = dbc.DATABASE_NAME

CHUNKSIZE = 4

In [ ]:
pathlist = list_datafile(path='./data')

plsqldb = Database(HOST, USER, PWD, DB)

logs = PipelineLogs("pipeline-prove", "pipeline.log").pipeline_logs()

pipeline(pathlist, plsqldb, CHUNKSIZE, logs)